In [1]:
import pandas as pd
import numpy as np
import utils as ut # esta librería tiene funciones para poder obtener un procesamiento del <T,H>

In [9]:
#arr = np.array([[1, 4, 2, 5, 3],[1, 4, 2, 5, 3]])
#indices = len(np.where(arr > 2.5)[0])
#print(indices)  # Salida: (array([1, 3]),)

6


In [2]:
import spacy
nlp = spacy.load("en_core_web_md") # modelo de nlp

/home/david/anaconda3/envs/rit/lib/python3.9/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
ut.load_vectors_in_lang(nlp,"./data/numberbatch-en-17.04b.txt") # carga de vectores en nlp.wv

In [96]:
textos = ["A brown and white dog is resting in the grass"]
hipotesis = ["A brown and white dog is running through the tall grass"]
clases = ["CONTRADICTION"]

In [110]:
for i in range(len(textos)):
    print(textos[i])
    print(hipotesis[i])
    t_lem_temp=ut.get_lemmas_(textos[i],nlp)
    h_lem_temp=ut.get_lemmas_(hipotesis[i],nlp)
    t_lem=[]
    h_lem=[]
    for a in t_lem_temp:
        if a not in t_lem:
            t_lem.append(a)
    for a in h_lem_temp:
        if a not in h_lem:
            h_lem.append(a)
    t_vectors=ut.get_matrix_rep2(t_lem, nlp, normed=False)
    h_vectors=ut.get_matrix_rep2(h_lem, nlp, normed=False)
    t_vectors_n=ut.get_matrix_rep2(t_lem, nlp, normed=True)
    h_vectors_n=ut.get_matrix_rep2(h_lem, nlp, normed=True)
    
    # # Obtencion de matriz de alineamiento, matriz de move earth y mutual information
    redondeo=1
    ma_n=np.dot(t_vectors_n,h_vectors_n.T)
    ma_n = np.clip(ma_n, 0, 1).round(redondeo)
    ma=pd.DataFrame(ma_n,index=t_lem,columns=h_lem)
    print(ma)
    print("Tokens de H: ",ma.columns)
    print("Tokens de T: ",ma.index)
    H_T_H = ut.entropia(ma.values.flatten())
    print("Entropia de ma: ",H_T_H)
    print(ma_n.shape)
    hs=np.array([])
    ent_anterior=0
    cambio=0
    for i in range(ma_n.shape[1]):
        print(ma.columns.values[i])
        hk=ma_n[:,i].flatten()
        print("Valores de",i, "dado T: ",hk)
        H_h_T = ut.entropia(ma_n[:,i].flatten())
        print("Entropia de",i, "dado T: ",H_h_T)
        print("Diferencia H(T,H)- H(h,T)",H_T_H-H_h_T)
        hs = np.append(hs,hk)
        ent_hs= ut.entropia(hs)
        if (ent_anterior>ent_hs):
            cambio+=1
        print(ent_anterior,"Entropia de ",hs, " incrementando h",i," :",ent_hs)
        ent_anterior=ent_hs
    print(cambio)
    # print("run",ut.entropia(ma[["run"]].values.flatten()))
    # print("brown",ut.entropia(ma[["brown"]].values.flatten()))
    # print("brown white",ut.entropia(ma[["brown","white"]].values.flatten()))
    # print("brown white dog",ut.entropia(ma[["brown","white","dog"]].values.flatten()))
    # print("brown white dog run",ut.entropia(ma[["brown","white","dog","run"]].values.flatten()))
    # print("brown white dog run tall",ut.entropia(ma[["brown","white","dog","run","tall"]].values.flatten()))
    # print("brown white dog run tall grass",ut.entropia(ma[["brown","white","dog","run","tall", "grass"]].values.flatten()))
    # print("brown white dog tall grass",ut.entropia(ma[["brown","white","dog","tall", "grass"]].values.flatten()))
    # print("brown white dog grass",ut.entropia(ma[["brown","white","dog","grass"]].values.flatten()))
    # print("-----------------------------------------")
    # for i in range(ma_n.shape[0]):
    #     print(ma.index.values[i])
    #     hk=ma_n[i].round(redondeo).flatten()
    #     print("Valores de",i, "dado H: ",hk)
    #     H_h_T = ut.entropia(ma_n[i].round(redondeo).flatten())
    #     print("Entropia de",i, "dado H: ",H_h_T)
    #     print("Diferencia H(T,H)- H(t,H)",H_T_H-H_h_T)
    

A brown and white dog is resting in the grass
A brown and white dog is running through the tall grass
       brown  white  dog  run  tall  grass
brown    1.0    0.5  0.1  0.0   0.2    0.1
white    0.5    1.0  0.0  0.0   0.2    0.1
dog      0.1    0.0  1.0  0.1   0.0    0.1
rest     0.0    0.0  0.0  0.1   0.0    0.0
grass    0.1    0.1  0.1  0.0   0.2    1.0
Tokens de H:  Index(['brown', 'white', 'dog', 'run', 'tall', 'grass'], dtype='object')
Tokens de T:  Index(['brown', 'white', 'dog', 'rest', 'grass'], dtype='object')
Entropia de ma:  2.039
(5, 6)
brown
Valores de 0 dado T:  [1.  0.5 0.1 0.  0.1]
Entropia de 0 dado T:  1.922
Diferencia H(T,H)- H(h,T) 0.11700000000000021
0 Entropia de  [1.  0.5 0.1 0.  0.1]  incrementando h 0  : 1.922
white
Valores de 1 dado T:  [0.5 1.  0.  0.  0.1]
Entropia de 1 dado T:  1.922
Diferencia H(T,H)- H(h,T) 0.11700000000000021
1.922 Entropia de  [1.  0.5 0.1 0.  0.1 0.5 1.  0.  0.  0.1]  incrementando h 1  : 1.971
dog
Valores de 2 dado T:  [0.1 0.  1.  

# Obtener stopwords:

In [3]:
def find_stopwords(frase,nlp):
    stops=set()
    doc = nlp(frase)
    for token in doc:
        if token.is_stop:
        #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
        #        token.shape_, token.is_alpha, token.is_stop)
            stops.add(token.lemma_)
    return stops

In [8]:
conjunto_stopwords=set()
for j in range(45):
    prueba=pd.read_csv("../OPENAI/data/SICK/TRAIN_"+str(j+1)+".csv")
    textos = prueba["sentence1"].to_list()       # almacenamiento en listas
    hipotesis = prueba["sentence2"].to_list()
    clases = prueba["gold_label"].to_list()
    for i in range(len(textos)):
        #print(textos[i])
        conjunto_stopwords.update(find_stopwords(textos[i],nlp))
        conjunto_stopwords.update(find_stopwords(hipotesis[i],nlp))

In [9]:
conjunto_stopwords

{"'s",
 'a',
 'about',
 'above',
 'across',
 'after',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'an',
 'and',
 'another',
 'around',
 'as',
 'at',
 'back',
 'be',
 'before',
 'behind',
 'below',
 'beside',
 'between',
 'both',
 'bottom',
 'by',
 'call',
 'can',
 'do',
 'down',
 'during',
 'each',
 'empty',
 'few',
 'five',
 'for',
 'four',
 'from',
 'front',
 'full',
 'go',
 'have',
 'he',
 'her',
 'himself',
 'his',
 'in',
 'into',
 'it',
 'its',
 'make',
 'many',
 'mostly',
 'next',
 'no',
 'nobody',
 'none',
 'not',
 'nothing',
 'of',
 'off',
 'on',
 'one',
 'only',
 'onto',
 'or',
 'other',
 'out',
 'over',
 'part',
 'put',
 'really',
 'same',
 'serious',
 'several',
 'she',
 'show',
 'side',
 'some',
 'someone',
 'something',
 'still',
 'that',
 'the',
 'their',
 'themselves',
 'there',
 'they',
 'this',
 'three',
 'through',
 'to',
 'together',
 'top',
 'toward',
 'towards',
 'two',
 'under',
 'up',
 'upon',
 'use',
 'various',
 'very',
 'via',
 'where',
 'which',
 'whi